In [1]:
%load_ext autoreload
%autoreload 2

# Imports and function definitions

In [ ]:
from pathlib import Path
import itertools
import json
from typing import Callable, Optional, Union

import gufe
import kartograf
import networkx as nx
import numpy as np
import openfe
from gufe import AtomMapper, LigandAtomMapping, LigandNetwork
from kartograf.filters import (
    filter_ringbreak_changes,
    filter_ringsize_changes,
    filter_whole_rings_only,
)
from lomap.graphgen import GraphGen
from rdkit import Chem
from tqdm import tqdm


# modified from openfe.ligand_network_planning to return the network instead of the graphml object
def generate_lomap_network(
    molecules: list[gufe.SmallMoleculeComponent],
    mappers: Union[AtomMapper, list[AtomMapper]],
    scorer: Callable,
    distance_cutoff: float = 0.4,
    max_path_length=6,
    actives: Optional[list[bool]] = None,
    max_dist_from_active=2,
    require_cycle_covering: bool = True,
    radial: bool = False,
    fast: bool = False,
    hub: Optional[gufe.SmallMoleculeComponent] = None,
) -> LigandNetwork:
    """Generate a LigandNetwork according to Lomap's network creation rules

    Parameters
    ----------
    molecules : list[SmallMoleculeComponent]
       molecules to map
    mappers : list[AtomMapper] or AtomMapper
       one or more Mapper functions to use to propose edges
    scorer: function
       scoring function for edges.  Should be a function which takes an AtomMapping and returns a value from 1.0 (best)
       to 0.0 (worst).  These values are use as the "distance" between two molecules, and compared against the
       'distance_cutoff' parameter
    distance_cutoff : float
       the maximum distance/dissimilarity between two molecules for an edge to be accepted
    max_path_length : int
      maximum distance between any two molecules in the resulting network
    actives : list[bool]
      for each molecule, if it is tagged as an active molecule
    max_dist_from_active
      when 'actives' is given, constrains the resulting map to
    require_cycle_covering : bool
      add cycles into the network
    radial : bool
      construct a radial/starmap network.  Note that this the map will not necessarily be a true radial map; edges
      will still obey the distance_cutoff and if 'require_cycle_covering' is true, this radial map will still
      feature cycles, default False
    fast : bool
      hmmm...
    hub : SmallMoleculeComponent, optional
      if radial, force this ligand to be the centre/hub of the radial graph
    """
    if not mappers:
        raise ValueError("At least one Mapper must be provided")
    if isinstance(mappers, gufe.AtomMapper):
        mappers = [mappers]
    if actives is None:
        actives = [False] * len(molecules)

    # gen n x n mappings with scores
    # initially all zero scores, i.e. impossible
    mtx = np.zeros((len(molecules), len(molecules)), dtype=float)
    # np array of mappings
    mps = np.zeros_like(mtx, dtype=object)

    # for all combinations of molecules
    for i, j in itertools.combinations(range(len(molecules)), 2):
        mA, mB = molecules[i], molecules[j]

        # pick best score across all mappings from all mappings
        best_mp: Optional[LigandAtomMapping] = None
        best_score = 0.0
        for mapper in mappers:
            mp: LigandAtomMapping
            score: float
            try:
                score, mp = max(
                    (scorer(mp), mp) for mp in (mapper.suggest_mappings(mA, mB))
                )
            except ValueError:
                # if mapper returned no mappings
                continue
            else:
                if score > best_score:
                    best_mp = mp
                    best_score = score

        if best_mp is None:
            # logger.debug(f"Found no mapping for {mA} {mB}")
            continue

        # logger.debug(f"Mapping for {mA} {mB} has score {best_score}")

        mtx[i, j] = mtx[j, i] = best_score
        mps[i, j] = mps[j, i] = best_mp.with_annotations({"score": best_score})

    gg = GraphGen(
        score_matrix=mtx,
        ids=list(range(mtx.shape[0])),
        names=[m.name for m in molecules],
        max_path_length=max_path_length,
        actives=actives,
        max_dist_from_active=max_dist_from_active,
        similarity_cutoff=1 - distance_cutoff,
        require_cycle_covering=require_cycle_covering,
        radial=radial,
        fast=fast,
        hub=hub.name if hub else None,
    )
    n: nx.Graph = gg.resultGraph
    return n


def gen_ligand_network(smcs):
    """
    Creates the Lomap ligand network using the KartografAtomMapper and
    the Lomap scorer.

    Parameters
    ----------
    smcs : list[SmallMoleculeComponents]
      List of SmallMoleculeComponents of the ligands.

    Returns
    -------
    openfe.LigandNetwork
      The Lomap generated LigandNetwork.
    """
    print("INFO: Generating Lomap Network")
    mapping_filters = [
        filter_ringbreak_changes,  # default
        filter_ringsize_changes,  # default
        filter_whole_rings_only,  # default
    ]
    mapper = kartograf.KartografAtomMapper(
        atom_map_hydrogens=True,
        additional_mapping_filter_functions=mapping_filters,
    )
    scorer = openfe.lomap_scorers.default_lomap_score
    ligand_network = generate_lomap_network(
        molecules=smcs, mappers=mapper, scorer=scorer
    )
    return ligand_network


def run_inputs(ligands):
    """
    Generate run json files for RBFE calculations

    Parameters
    ----------
    ligands : pathlib.Path
      A Path to a ligands SDF.
    pdb : pathlib.Path
      A Path to a protein PDB file.
    cofactors : Optional[pathlib.Path]
      A Path to an SDF file containing the system's cofactors.
    output: pathlib.Path
      A Path to a directory where the transformation json files
      and ligand network graphml file will be stored into.
    """
    # Create the small molecule components of the ligands
    rdmols = [mol for mol in Chem.SDMolSupplier(str(ligands), removeHs=False)]
    smcs = [openfe.SmallMoleculeComponent.from_rdkit(mol) for mol in rdmols]

    # Create ligand network
    ligand_network = gen_ligand_network(smcs)

    # # Store the ligand network as a graphml file
    # with open(output / "ligand_network.graphml", mode="w") as f:
    #     f.write(ligand_network.to_graphml())

    return ligand_network


def reformat_network(network: nx.Graph):
    result = {"nodes": [], "edges": []}
    for node, data in network.nodes(data=True):
        node_dict = {"id": node}
        node_dict.update(data)
        result["nodes"].append(node_dict)

    # Add edges to the result
    for source, target, data in network.edges(data=True):
        edge_dict = {"source": source, "target": target}
        edge_dict.update(data)
        result["edges"].append(edge_dict)

    new_nodes = {node["fname_comp"]: {} for node in result["nodes"]}

    # Create a mapping of old IDs to new names
    id_to_name = {node["id"]: node["fname_comp"] for node in result["nodes"]}

    # Transform edges
    new_edges = []
    for edge in result["edges"]:
        new_edge = {
            "from": id_to_name[edge["source"]],
            "to": id_to_name[edge["target"]],
            "similarity": edge["similarity"],
            "strict_flag": edge["strict_flag"],
        }
        new_edges.append(new_edge)

    # Create the new data structure
    new_data = {"nodes": new_nodes, "edges": new_edges}
    return new_data

## JACS dataset

In [ ]:
JACS_targets = [
    "bace",
    "p38",
    "thrombin",
    "cdk2",
    "jnk1",
    "ptp1b",
    "tyk2",
    "mcl1",
]

for target in tqdm(JACS_targets):
    # if Path(f"../perturbations/{target}/mapping.json").exists():
    #     print(f"Skipping {target} as mapping already exists")
    #     continue
    rootpath = Path().absolute()
    network = run_inputs(ligands=rootpath / f"{target}/ligands/ligands.sdf")
    new_data = reformat_network(network)
    Path(f"../perturbations/{target}/mapping.json").write_text(
        json.dumps(new_data, indent=2)
    )

  0%|          | 0/8 [00:00<?, ?it/s]

INFO: Generating Lomap Network


 12%|█▎        | 1/8 [00:43<05:04, 43.48s/it]

INFO: Generating Lomap Network


 25%|██▌       | 2/8 [01:22<04:06, 41.10s/it]

...

 75%|███████▌  | 6/8 [02:00<00:30, 15.20s/it]

INFO: Generating Lomap Network


 88%|████████▊ | 7/8 [02:07<00:12, 12.26s/it]

INFO: Generating Lomap Network


100%|██████████| 8/8 [03:00<00:00, 22.60s/it]


## Merck dataset

In [3]:
merck_targets = [
    "cdk8",
    "cmet",
    "eg5",
    "hif2a",
    # "pfkfb3", # No data for this yet 🥲
    "shp2",
    "syk",
    "tnks2",
]

for target in tqdm(merck_targets):
    if target == "pfkfb3":
        print("Skipping PFKFB3 (not yet in the repo)")
        continue
    # if Path(f"../perturbations/{target}/mapping.json").exists():
    #     print(f"Skipping {target} as mapping already exists")
    #     continue
    rootpath = Path().absolute()
    network = run_inputs(ligands=rootpath / f"{target}/ligands/ligands.sdf")
    new_data = reformat_network(network)
    Path(f"../perturbations/{target}/mapping.json").write_text(
        json.dumps(new_data, indent=2)
    )

  0%|          | 0/7 [00:00<?, ?it/s]

INFO: Generating Lomap Network


 14%|█▍        | 1/7 [00:51<05:09, 51.65s/it]

INFO: Generating Lomap Network


 29%|██▊       | 2/7 [01:16<02:59, 35.97s/it]

...

 71%|███████▏  | 5/7 [03:27<01:22, 41.24s/it]

INFO: Generating Lomap Network


 86%|████████▌ | 6/7 [05:07<01:01, 61.01s/it]

INFO: Generating Lomap Network


100%|██████████| 7/7 [05:37<00:00, 48.18s/it]


# Incorporate energies

In [3]:
def add_ddg_to_edges(
    edges_list: list, nodes_dict: dict, dg_key: str, dgerr_key: Optional[str] = None
):
    in_edge_nodes = []
    all_nodes = list(nodes_dict.keys())
    for edge in edges_list:
        _from = nodes_dict[edge["from"]]
        _to = nodes_dict[edge["to"]]
        edge.update(
            {
                "ddg_value": round(_to[dg_key] - _from[dg_key], 2),
            }
        )
        if dgerr_key is not None:
            edge.update(
                {
                    "ddg_error": (
                        round(
                            np.sqrt(
                                np.power(_to[dgerr_key], 2.0)
                                + np.power(_from[dgerr_key], 2.0)
                            ),
                            2,
                        )
                        if _to[dgerr_key] is not None and _from[dgerr_key] is not None
                        else None
                    )
                }
            )
        in_edge_nodes.extend([edge["to"], edge["from"]])
    not_in_edge_nodes = np.setdiff1d(all_nodes, in_edge_nodes).tolist()
    if not_in_edge_nodes:
        print(f"Nodes not in edges: {not_in_edge_nodes}")
    return edges_list

In [7]:
from QligFEP.chemIO import MoleculeIO

merck_targets = [
    "cdk8",
    "cmet",
    "eg5",
    "hif2a",
    # "pfkfb3", # No data for this yet 🥲
    "shp2",
    "syk",
    "tnks2",
]
JACS_targets = [
    "bace",
    "p38",
    "thrombin",
    "cdk2",
    "jnk1",
    "ptp1b",
    "tyk2",
    "mcl1",
]

for target in JACS_targets + merck_targets:

    network_mapping = json.loads(
        Path(f"../perturbations/{target}/mapping.json").read_text()
    )

    molio = MoleculeIO(f"{target}/ligands/ligands.sdf")
    target_dgs = {}

    for molecule, name in zip(molio.molecules, molio.lig_names):
        network_mapping["nodes"][name]["dg_value"] = molecule.properties["r_exp_dg"]
        network_mapping["nodes"][name]["smiles"] = molecule.to_smiles()
        network_mapping["nodes"][name][
            "formal_charge"
        ] = molecule.total_charge.magnitude

    network_mapping["edges"] = add_ddg_to_edges(
        network_mapping["edges"], network_mapping["nodes"], dg_key="dg_value"
    )
    for edge in network_mapping["edges"]:
        _from_char = network_mapping["nodes"][edge["from"]]["formal_charge"]
        _to_charg = network_mapping["nodes"][edge["to"]]["formal_charge"]
        same_charge = _from_char == _to_charg
        edge.update({"same_charge": same_charge})

    Path(f"../perturbations/{target}/mapping.json").write_text(
        json.dumps(network_mapping, indent=2)
    )  # bace flip edges are incorporated manually in the json file

2024-12-10 14:32:46 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 2gg. Reindexed.
2024-12-10 14:32:46 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 2q. Reindexed.
2024-12-10 14:32:46 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 2t. Reindexed.
2024-12-10 14:32:47 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 2l. Reindexed.
2024-12-10 14:32:47 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 2ee. Reindexed.
2024-12-10 14:32:47 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 2aa. Reindexed.
2024-12-10 14:32:47 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 2o. Reindex